# Gymnasium integration
[Gymnasium](https://gymnasium.farama.org/) is an API standard for reinforcement learning with a diverse collection of reference environments. Stormvogel supports some integration with gymnasium. In particular, you can construct explicit models from the gymnasium environmnents under Gymnasium's [ToyText](https://gymnasium.farama.org/environments/toy_text/) (except Blackjack). 

## FrozenLake
Let us create one of these environments, called FrozenLake.
Our agent wants to get to the present. Currently, it just chooses a random action.

In [3]:
import gymnasium as gym
from stormvogel.extensions.gym_grid import *

env = gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery=False)  # Set `is_slippery=True` for stochastic behavior
#embed_video(create_video(env, "./video", None))

We can convert it into an explicit MDP as follows. Each state has a label that relates to the coordinates of the tile.

In [4]:
from stormvogel.show import show
from stormvogel.layout import Layout
sv_model = gymnasium_to_stormvogel(env)
vis = show(sv_model, layout=Layout("layouts/frozenlake.json"))

Output()

<IPython.core.display.Javascript object>

Output()

Now, let's do some model checking to calculate a strategy to solve the puzzle. We will tell the model checker to maximize the probability of getting to the target state (the present).

In [5]:
from stormvogel.model_checking import model_checking
target = get_target_state(env)
res = model_checking(sv_model, f'Pmax=? [F "target"]')
vis2 = show(sv_model, result=res, layout=Layout("layouts/frozenlake.json"))

Output()

Output()

Let's highlight the path to see what the scheduler is doing.

In [6]:
from stormvogel.simulator import simulate_path
path = simulate_path(sv_model, scheduler=res.scheduler, steps=20)
vis2.highlight_path(path, color="orange")

Alternatively, we can show what our scheduler does in the frozen lake environment itself.

In [7]:
from stormvogel.extensions.gym_grid import *
embed_video(create_video_scheduler(sv_model, env, "./video", res.scheduler))

We can also define a function to act as the scheduler.

In [8]:
def my_scheduler(s: stormvogel.model.State):
    # "←" "↓" "→" "↑"
    return stormvogel.model.Action.create("→")

embed_video(create_video_scheduler(sv_model, env, "./video", my_scheduler))

## CliffWalking
CliffWalking is a slightly more boring version of FrozenLake. You can apply the same principles that we just applied to FrozenLake.

In [9]:
import gymnasium as gym
from stormvogel.extensions.gym_grid import *

env = gym.make("CliffWalking-v0", render_mode="rgb_array")
#embed_video(create_video(env, "./video", None))

In [10]:
from stormvogel.show import show
from stormvogel.layout import Layout

sv_model = gymnasium_to_stormvogel(env)
vis = show(sv_model, layout=Layout("layouts/cliffwalking.json"))

Output()

Output()

In [11]:
from stormvogel.model_checking import model_checking
target = get_target_state(env)
res = model_checking(sv_model, f'Pmax=? [F "target"]')
embed_video(create_video_scheduler(sv_model, env, "./video", res.scheduler))

## Taxi

In [10]:
import gymnasium as gym
from stormvogel.extensions.gym_grid import *

env = gym.make("Taxi-v3", render_mode="rgb_array")  # Set `is_slippery=True` for stochastic behavior
#embed_video(create_video(env, "./video", None))

In [11]:
sv_model = gymnasium_to_stormvogel(env)
# This model is so big that it is better not to display it.
sv_model.summary()

/home/ivo/git/stormvogel/env/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:416: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")


'ModelType.MDP model with name None, 126 states, 6 actions, and 252 distinct labels.'

In [9]:
from stormvogel.model_checking import model_checking

sv_model = gymnasium_to_stormvogel(env)
target = get_target_state(env)
res = model_checking(sv_model, f'Rmax=? [S]')
embed_video(create_video_scheduler(sv_model, env, "./video", res.scheduler, limit=20))

222 []


IndexError: list index out of range